In [12]:
import pvl, struct, quaternion
import numpy as np

In [2]:
def read_position_table(label, file):
    with open(file, 'rb') as f:
        f.seek(int(label['StartByte']))
        data = f.read(int(label['Bytes']))
    field_count = len(label.getlist('Field'))
    record_count = int(label['Records'])
    format = 'd' * field_count
    positions = []
    velocities = []
    times = []
    for index in range(record_count):
        doubles = struct.unpack_from(format, data, index * field_count * 8)
        positions.append(doubles[:3])
        if field_count > 3:
            times.append(doubles[-1])
        if field_count == 7:
            velocities.append(doubles[3:6])
    return positions, velocities, times

In [26]:
def read_rotation_table(label, file):
    with open(file, 'rb') as f:
        f.seek(int(label['StartByte']))
        data = f.read(int(label['Bytes']))
    field_count = len(label.getlist('Field'))
    record_count = int(label['Records'])
    format = 'd' * field_count
    rotations = []
    velocities = []
    times = []
    for index in range(record_count):
        doubles = struct.unpack_from(format, data, index * field_count * 8)
        # Euler angle functions
        if field_count == 3:
            rotations.append(doubles[:3])
        else:
            rotations.append(np.quaternion(doubles[0], doubles[1], doubles[2], doubles[3]))
            times.append(doubles[-1])
            if field_count == 8:
                velocities.append(doubles[4:7])
                
    const_rotation_mat = np.array(label['ConstantRotation'])
    const_rotation = quaternion.from_rotation_matrix(np.reshape(const_rotation_mat, (3, 3)))
    return rotations, velocities, times, const_rotation

In [4]:
cube_file = '/Users/jmapel/eal/cubes/CAS-MCO-2016-11-22T16.38.39.354-NIR-02036-00.cub'

In [5]:
label = pvl.load(cube_file)
tables = label.getlist('Table')
print(tables)

[PVLObject([
  ('Name', 'InstrumentPointing')
  ('StartByte', 336115)
  ('Bytes', 64)
  ('Records', 1)
  ('ByteOrder', 'Lsb')
  ('TimeDependentFrames', [-143410, -143400, -143000, 1])
  ('ConstantFrames', [-143420, -143410])
  ('ConstantRotation',
   [6.12323399573677e-17,
    0.0,
    1.0,
    0.98513931843935,
    0.17175716364932,
    -6.03223856520476e-17,
    -0.17175716364932,
    0.98513931843935,
    1.05170930346882e-17])
  ('CkTableStartTime', 533104787.53776)
  ('CkTableEndTime', 533104787.53776)
  ('CkTableOriginalSize', 1)
  ('FrameTypeCode', 3)
  ('Description', 'Created by spiceinit')
  ('Kernels',
   ['$tgo/kernels/ck/em16_tgo_sc_fpp_d161102_pointing_20161028_20171129_s20160719_v01.bc',
    '$tgo/kernels/ck/em16_tgo_cassis_ipp_tel_20160407_20170309_s20170116_v02.bc',
    '$tgo/kernels/fk/em16_tgo_v12.tf',
    '$tgo/kernels/fk/rssd0002.tf'])
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q0')
    ('Type', 'Double')
    ('Size', 1)
  ]))
  ('Field',
   PVLGroup([
    ('Name

In [6]:
read_position_table(tables[1], cube_file)

([(4.743092590963316e-305, 4.739987214924918e-305, 9.462663060940373e+251)],
 [(-1.6878690421469897e+271, 5.29682553422139e-228, -5.8500298023031387e-98)],
 [-1.1371270199850502e-143])

In [27]:
read_rotation_table(tables[0], cube_file)

([quaternion(-1.34337767237385e-225, -5.29807286672666e-110, 8.1011017870871e-115, -1.44413722000277e+194)],
 [(-8089.420679908819, -3.5340714299727612e+249, -2.2489288021413767e-100)],
 [1.9746554944903616e-307],
 quaternion(0.54123866354163, 0.4550392390637, 0.54123866354163, 0.4550392390637))

In [28]:
read_rotation_table(tables[3], cube_file)

KeyError: 'ConstantRotation'